In [1]:
# importing the modules 

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt

%matplotlib inline

In [3]:
#loading the datasets provided "train.csv, test.csv"

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
# 1. If for any column(s), the variance is equal to zero, then you need to remove those variable(s).

## checking for thw colummns with variance 0

In [6]:
train_df.var()[train_df.var()==0].index.values

array(['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290',
       'X293', 'X297', 'X330', 'X347'], dtype=object)

In [7]:
cols = [c for c in train_df.columns if 'X' in c]
print('Number of features: {}'.format(len(cols)))

Number of features: 376


In [8]:
# checking for the unoque values "Constant features"

counts = [[]]
for c in cols:
    typ = train_df[c].dtype
    uniq = len(np.unique(train_df[c]))
    if uniq == 1:
        counts[0].append(c)


In [9]:

print('Constant features:', counts[0])


Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']


In [10]:
# Now we are removing teh columns which we found having 0 variane which are constance variables

In [11]:
train_df = train_df.drop(['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347'], axis = 1)

In [12]:
#removing unique values in test_df

In [13]:
counts = [[]]
for c in cols:
    typ = test_df[c].dtype
    uniq = len(np.unique(test_df[c]))
    if uniq == 1:
        counts[0].append(c)

In [14]:
print('Constant features:', counts[0])

Constant features: ['X257', 'X258', 'X295', 'X296', 'X369']


In [15]:
test_df = test_df.drop(['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347'], axis = 1)

In [16]:
def check_missing_values(df):
    if df.isnull().any().any():
        print("There are missing values in the dataframe")
    else:
        print("There are no missing values in the dataframe")
check_missing_values(train_df)
check_missing_values(test_df)

There are no missing values in the dataframe
There are no missing values in the dataframe


In [17]:
# Removing target variable from train dataset and 'ID'from both train and test data sets

In [19]:
x_train = train_df.drop(['y','ID'],axis=1)

In [20]:
x_test = test_df.drop(['ID'],axis=1)

In [21]:
# Applying lable encoder for both x_test and y_test

In [22]:
le = LabelEncoder()

In [23]:
x_train.shape

(4209, 364)

In [24]:
x_test.shape

(4209, 364)

In [26]:
for c in x_train.columns:
    if x_train[c].dtype == 'object':
        le.fit(list(x_train[c].values) + list(x_test[c].values)) 
        x_train[c] = le.transform(list(x_train[c].values))
        x_test[c] = le.transform(list(x_test[c].values))

In [27]:
x_train.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,37,23,20,0,3,27,9,14,0,0,...,0,0,1,0,0,0,0,0,0,0
1,37,21,22,4,3,31,11,14,0,0,...,1,0,0,0,0,0,0,0,0,0
2,24,24,38,2,3,30,9,23,0,0,...,0,0,0,0,0,0,1,0,0,0
3,24,21,38,5,3,30,11,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,24,23,38,5,3,14,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
x_test.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,24,23,38,5,3,26,0,22,0,0,...,0,0,0,1,0,0,0,0,0,0
1,46,3,9,0,3,9,6,24,0,0,...,0,0,1,0,0,0,0,0,0,0
2,24,23,19,5,3,0,9,9,0,0,...,0,0,0,1,0,0,0,0,0,0
3,24,13,38,5,3,32,11,13,0,0,...,0,0,0,1,0,0,0,0,0,0
4,49,20,19,2,3,31,8,12,0,0,...,1,0,0,0,0,0,0,0,0,0


In [29]:
y_train = train_df['y'].values
id_test = test_df['ID'].values

In [30]:
y_train

array([130.81,  88.53,  76.26, ..., 109.22,  87.48, 110.85])

In [31]:
# Performing dimentionality reduction on both of the dataframes 

In [33]:
from sklearn.decomposition import PCA

In [34]:
n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

In [35]:
# train test splitting and using xgboost

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
x_train, x_valid, y_train, y_valid = train_test_split(
        pca2_results_train, 
        y_train, test_size=0.2, 
        random_state=42)

In [38]:
import xgboost as xgb

In [39]:

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(pca2_results_test)

In [40]:
params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

In [41]:
def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

In [42]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [43]:
clf = xgb.train(params, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)


[11:44:41] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:98.99708	valid-rmse:98.88793	train-r2:-59.49748	valid-r2:-61.82575
Multiple eval metrics have been passed: 'valid-r2' will be used for early stopping.

Will train until valid-r2 hasn't improved in 50 rounds.
[10]	train-rmse:81.14501	valid-rmse:81.05737	train-r2:-39.64585	valid-r2:-41.21202
[20]	train-rmse:66.59513	valid-rmse:66.54775	train-r2:-26.37647	valid-r2:-27.45232
[30]	train-rmse:54.75217	valid-rmse:54.74006	train-r2:-17.50526	valid-r2:-18.25137
[40]	train-rmse:45.12996	valid-rmse:45.13305	train-r2:-11.57252	valid-r2:-12.08701
[50]	train-rmse:37.33654	valid-rmse:37.35689	train-r2:-7.60519	valid-r2:-7.96587
[60]	train-rmse:31.04985	valid-rmse:31.07902	train-r2:-4.95130	valid-r2:-5.20563
[70]	train-rmse:26.00603	valid-rmse:26.03417	train-r2:-3.17485	valid-r2:-3.35450
[80]	train-rmse:21.97161	valid-rmse:22.00452	train-r2:-1.98000	valid-r2:-2.

In [44]:
# predicting the test dataframe using XGBoost

In [45]:
p_test = clf.predict(d_test)

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test

In [47]:
sub.head()

,ID,y
0,1,76.275734
1,2,95.872688
2,3,81.178894
3,4,77.383751
4,5,112.213394
